# SQL. Анализ сервиса чтения книг по подписке

## Предистория

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

## Цели и задачи проекта

Первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

![alt text](shema.png)

In [9]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [10]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [11]:
# функция, кторая будет вывадить результат запроса на экран
def result(query):
    return pd.io.sql.read_sql(query, con = engine)

#### Исследование тапблиц

In [4]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''SELECT * FROM authors LIMIT 5'''
result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''SELECT * FROM publishers LIMIT 5'''
result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = '''SELECT * FROM ratings LIMIT 5'''
result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = '''SELECT * FROM reviews LIMIT 5'''
result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


#### Количество книг опубликованных после 1 января 2000 года

In [9]:
query = '''
           SELECT COUNT(*) 
           FROM books 
           WHERE publication_date::date > '2000-01-01'
           '''
result(query)

,count
0,819


После 1 января 2020 года было опубликовано 819 книг

#### Количество обзоров и средняя оценка для каждой книги

In [10]:
query = '''
           SELECT b.title, b.num_pages, b.publication_date, COUNT(DISTINCT r.review_id) as count_of_rewiews, ROUND(AVG(rt.rating), 2) as avg_rating
           FROM books AS b
           LEFT JOIN reviews AS r ON b.book_id=r.book_id
           LEFT JOIN ratings AS rt ON b.book_id=rt.book_id
           GROUP BY b.book_id
           ORDER BY count_of_rewiews DESC
           '''
result(query)

,title,num_pages,publication_date,count_of_rewiews,avg_rating
0,Twilight (Twilight #1),501,2006-09-06,7,3.66
1,Water for Elephants,335,2007-05-01,6,3.98
2,The Glass Castle,288,2006-01-17,6,4.21
3,Harry Potter and the Prisoner of Azkaban (Harr...,435,2004-05-01,6,4.41
4,The Curious Incident of the Dog in the Night-Time,226,2004-05-18,6,4.08
...,...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,404,1991-11-30,0,3.67
996,The Natural Way to Draw,240,1990-02-01,0,3.00
997,The Cat in the Hat and Other Dr. Seuss Favorites,61,2003-10-14,0,5.00
998,Essential Tales and Poems,688,2004-10-25,0,4.00


Максимальное количество обзоров для одной книги - 7, минимальное 0. Максимальный средний рейтинг для книги - 5 

#### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
query = '''
           SELECT COUNT(DISTINCT b.book_id), b.publisher_id, p.publisher
           FROM books AS b
           LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
           WHERE b.num_pages > 50
           GROUP BY b.publisher_id, p.publisher
           ORDER BY COUNT(DISTINCT b.book_id) DESC
           LIMIT 1
           '''
result(query)

,count,publisher_id,publisher
0,42,212,Penguin Books


Вывод: Наибольшеее количество книг (42) выпустило издательство Penguin Books.

#### Автор с самой высокой средней оценкой книг  (учитываются только книги с 50 и более оценками)

In [12]:
query = '''
           SELECT a.author, ROUND(AVG(rt.rating), 2) as avg_rating
           FROM books AS b
           LEFT JOIN authors AS a ON b.author_id=a.author_id
           LEFT JOIN ratings AS rt ON b.book_id=rt.book_id
           WHERE b.book_id IN (
                  SELECT book_id
                  FROM ratings 
                  GROUP BY book_id
                  HAVING COUNT(rating_id) >= 50)
           GROUP BY a.author
           ORDER BY avg_rating DESC
           LIMIT 1
           '''
result(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Вывод: J.K. Rowling/Mary GrandPré является автором с самым высоким средним рейтингом книг - 4,29

#### Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [13]:
query = ''' SELECT ROUND(AVG(count_rew), 2) AS avg_count_rew
            FROM 
            (SELECT COUNT(username) as count_rew
            FROM reviews
            GROUP BY username
            HAVING username IN
               (SELECT username 
               FROM ratings 
               GROUP BY username
               HAVING COUNT(username) > 48)) as one 
        
           '''
result(query)

,avg_count_rew
0,24.0


Вывод: Среднее количество обзоров от пользователей, которые поставили более 48 оценок - 24.

## Общий вывод

Приобретенный сервис хранит 1000 книг 
- После 1 января 2020 года было опубликовано 819 книг
- Максимальное количество обзоров для одной книги - 7, минимальное 0. Максимальный средний рейтинг для книги - 5
- Наибольшеее количество книг (42) выпустило издательство Penguin Books.
- J.K. Rowling/Mary GrandPré является автором с самым высоким средним рейтингом книг - 4,29
- Среднее количество обзоров от пользователей, которые поставили более 48 оценок - 24.